# snowflakeR Quickstart -- Local Environment

This notebook is for **local R environments** (RStudio, Posit Workbench, JupyterLab with R kernel).
For Snowflake Workspace Notebooks, use `workspace_quickstart.ipynb`.

**Before you start:** Copy `notebook_config.yaml.template` to `notebook_config.yaml`
and edit it with your account, warehouse, database, and schema.

**Sections:**
1. Setup
2. Connect & set execution context
3. Queries & Table Operations
4. DBI & dbplyr Integration
5. Visualization with ggplot2
6. Cleanup

## 1. Setup

Install snowflakeR and set up the Python environment (one time only).

```r
# install.packages("pak")
# pak::pak("Snowflake-Labs/snowflakeR")
# snowflakeR::sfr_install_python_deps()
```

In [ ]:
library(snowflakeR)

---
## 2. Connect & Set Execution Context

`sfr_load_notebook_config()` reads `notebook_config.yaml` and runs
`USE WAREHOUSE / DATABASE / SCHEMA` to set the execution context.

All table references use fully qualified names via `sfr_fqn()` for
consistency with Workspace Notebooks.

In [ ]:
# Connect (reads ~/.snowflake/connections.toml or config connection params)
conn <- sfr_connect()

# Load config and set execution context
conn <- sfr_load_notebook_config(conn)
conn

---
## 3. Queries & Table Operations

In [ ]:
# Run a SQL query
result <- sfr_query(conn, "SELECT CURRENT_TIMESTAMP() AS now, CURRENT_USER() AS user_name")
result

In [ ]:
# Write a data.frame to Snowflake (fully qualified name)
sfr_write_table(conn, sfr_fqn(conn, "SFR_MTCARS"), mtcars, overwrite = TRUE)

In [ ]:
# List tables
tables <- sfr_list_tables(conn)
head(tables, 10)

In [ ]:
# Read it back (fully qualified name)
df <- sfr_read_table(conn, sfr_fqn(conn, "SFR_MTCARS"))
head(df, 5)

In [ ]:
# Describe columns
sfr_list_fields(conn, sfr_fqn(conn, "SFR_MTCARS"))

---
## 4. DBI & dbplyr Integration

In [ ]:
library(DBI)

DBI::dbGetQuery(conn, "SELECT 42 AS answer")
DBI::dbExistsTable(conn, sfr_fqn(conn, "SFR_MTCARS"))

In [ ]:
library(dplyr)
library(dbplyr)

# Lazy reference to Snowflake table (fully qualified)
cars_tbl <- tbl(conn, I(sfr_fqn(conn, "SFR_MTCARS")))

# dplyr pipeline -- generates SQL, runs on collect()
summary <- cars_tbl |>
  group_by(cyl) |>
  summarise(
    n       = n(),
    avg_mpg = mean(mpg, na.rm = TRUE),
    avg_hp  = mean(hp, na.rm = TRUE)
  ) |>
  arrange(cyl) |>
  collect()

summary

---
## 5. Visualization with ggplot2

In [ ]:
library(ggplot2)

df <- sfr_read_table(conn, sfr_fqn(conn, "SFR_MTCARS"))

ggplot(df, aes(x = wt, y = mpg, color = factor(cyl))) +
  geom_point(size = 3) +
  labs(title = "MPG vs Weight by Cylinder Count",
       x = "Weight (1000 lbs)", y = "Miles per Gallon",
       color = "Cylinders") +
  theme_minimal()

---
## 6. Cleanup

In [ ]:
# Uncomment to clean up demo objects
# (commented out to avoid accidental deletion on Run All)
#
# sfr_execute(conn, paste("DROP TABLE IF EXISTS", sfr_fqn(conn, "SFR_MTCARS")))
# cat("Cleanup complete.\n")